In [90]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.autograd as autograd

import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import os
import numpy as np
from model import *
from common_net import *

In [136]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
    
opt={}
opt['batchSize']=100
opt['dropout']=0.0
opt['ndres']=16
opt['ngres']=32
opt['ngres_gate']=32
opt['ngf']=16
opt['ngf_gate']=32
opt['ndf']=4
opt['out_dim']=1
opt['nz']=10
opt['ngpu']=1
opt['num_samples']=1000000
opt['outf']='./analysis_conv_gated_resnet_resnet_gen_18/'
opt['ndiscrete']=10
opt['ncontinuous']=2
opt['nnoise']=62
opt['nsalient']=opt['ndiscrete'] + opt['ncontinuous']
opt=dotdict(opt)
os.mkdir(opt.outf)

In [133]:
 # fixed random variables
c = np.linspace(-1, 1, opt.ndiscrete).reshape(1, -1)
c = np.repeat(c, opt.ndiscrete, 0).reshape(-1, 1)

c1 = np.hstack([c, np.zeros_like(c)])
c2 = np.hstack([np.zeros_like(c), c])

idx = np.arange(opt.ndiscrete).repeat(opt.ndiscrete)
one_hot = np.zeros((opt.ndiscrete*opt.ndiscrete, opt.ndiscrete))
one_hot[range(opt.ndiscrete*opt.ndiscrete), idx] = 1
fix_noise = torch.Tensor(opt.ndiscrete*opt.ndiscrete, opt.nnoise).uniform_(-1, 1)

In [112]:
dis_c = torch.FloatTensor(opt.batchSize, opt.ndiscrete).cuda()
con_c = torch.FloatTensor(opt.batchSize, opt.ncontinuous).cuda()
noise = torch.FloatTensor(opt.batchSize, opt.nnoise).cuda()

dis_c = Variable(dis_c)
con_c = Variable(con_c)
noise = Variable(noise)

In [135]:
print(opt)
g = GatedResnetConvResnetG(opt)
g.cuda()
#load generator
gen_path='conv_gated_resnet_resnet_gen_18/netG_epoch_25.pth'
g.load_state_dict(torch.load(gen_path))

{'ngres': 32, 'dropout': 0.0, 'ndiscrete': 10, 'nsalient': 12, 'ngres_gate': 32, 'ndres': 16, 'ngpu': 1, 'out_dim': 1, 'ngf_gate': 32, 'nnoise': 62, 'ngf': 16, 'batchSize': 100, 'ndf': 4, 'nz': 10, 'ncontinuous': 2, 'outf': './analysis_conv_gated_resnet_resnet_gen_17/', 'num_samples': 1000000}


In [23]:
g.

Sequential (
  (0): Linear (12 -> 32)
  (1): ReLU ()
  (2): ResBlock (
    (model): Sequential (
      (0): Linear (32 -> 32)
      (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True)
      (2): ReLU (inplace)
      (3): Linear (32 -> 32)
    )
  )
  (3): ResBlock (
    (model): Sequential (
      (0): Linear (32 -> 32)
      (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True)
      (2): ReLU (inplace)
      (3): Linear (32 -> 32)
    )
  )
  (4): ResBlock (
    (model): Sequential (
      (0): Linear (32 -> 32)
      (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True)
      (2): ReLU (inplace)
      (3): Linear (32 -> 32)
    )
  )
  (5): ResBlock (
    (model): Sequential (
      (0): Linear (32 -> 32)
      (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True)
      (2): ReLU (inplace)
      (3): Linear (32 -> 32)
    )
  )
  (6): ResBlock (
    (model): Sequential (
      (0): Linear (32 -> 32)
      (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=Tru

In [137]:
dis_c.data.copy_(torch.Tensor(one_hot))
con_c.data.copy_(torch.from_numpy(c2))

z_gate=torch.cat([dis_c,con_c],1)

In [138]:
input_gate = z_gate.cuda()
output_gate = g.gate(input_gate)

In [139]:
def thresh(x,val):
    if x<val:
        return 0
    else:
        return x

In [98]:
print(output_gate[0])

Variable containing:
 0.0254
 0.0195
 0.0211
 0.0567
 0.0130
 0.0138
 0.0638
 0.0391
 0.1290
 0.1400
 0.0230
 0.0222
 0.0223
 0.0856
 0.0711
 0.1482
 0.9616
 0.0963
 0.1334
 0.1362
 0.8554
 0.3667
 0.2813
 0.9964
 0.9982
 0.9729
 0.1750
 0.9994
 0.2348
 0.0892
 0.9993
 0.0158
[torch.cuda.FloatTensor of size 32 (GPU 0)]



In [99]:
print(output_gate[10])

Variable containing:
 0.0064
 0.0481
 0.0034
 0.0062
 0.0021
 0.0015
 0.0034
 0.0030
 0.0309
 0.1794
 0.0065
 0.0043
 0.0282
 0.0704
 0.0437
 0.1641
 0.9942
 0.1386
 0.0930
 0.1561
 0.9803
 0.3115
 0.2829
 0.9998
 0.9988
 0.9706
 0.1850
 0.9997
 0.2313
 0.0849
 0.9992
 0.0182
[torch.cuda.FloatTensor of size 32 (GPU 0)]



In [100]:
print(output_gate[20])

Variable containing:
 0.0033
 0.0054
 0.0052
 0.0067
 0.0705
 0.0017
 0.0192
 0.0380
 0.1311
 0.1818
 0.0224
 0.0048
 0.0267
 0.0790
 0.1665
 0.1484
 0.9941
 0.0477
 0.1233
 0.0832
 0.9628
 0.3353
 0.2544
 0.9996
 0.9995
 0.9871
 0.1694
 0.9999
 0.2441
 0.0910
 0.9989
 0.0163
[torch.cuda.FloatTensor of size 32 (GPU 0)]



In [143]:
def thresh(output_gate,thresh_val):
    output_gate_tensor = output_gate.data
    thresholded = output_gate_tensor.le(thresh_val)
    print(thresholded.size())
    for i in range(10):
        print(torch.sum(thresholded[i*10:(i+1)*10,:]))
    print(torch.sum(output_gate_tensor.le(thresh_val)))
    output_gate_tensor.masked_fill_(output_gate_tensor.le(thresh_val),0)
    return Variable(output_gate_tensor)

In [128]:
def value_aware_thresh(output_gate,thresh_val):
    
    noise.data.copy_(fix_noise)
    input_main = noise
    input_main = input_main.resize(opt.batchSize,opt.nnoise,1,1)                            
        
    output = g.main_initial(input_main)    
    for i in range(opt.ngres):
        alpha = output_gate[:,i]
        alpha = alpha.resize(opt.batchSize,1,1,1)                                                
        temp_output=g.main[i](output,alpha)
            
        for j in range(opt.batchSize):
            
        print(output.size())
        print(alpha.size())
        
        output = temp_output
    #output_gate_tensor = output_gate.data
    #thresholded = output_gate_tensor.le(thresh_val)
    #print(thresholded.size())
    #for i in range(10):
    #    print(torch.sum(thresholded[i*10:(i+1)*10,:]))
    #print(torch.sum(output_gate_tensor.le(thresh_val)))
    #output_gate_tensor.masked_fill_(output_gate_tensor.le(thresh_val),0)
    #return Variable(output_gate_tensor)

In [116]:
output_gate[20]

Variable containing:
 0.0000
 0.0000
 0.0000
 0.0000
 0.0705
 0.0000
 0.0000
 0.0000
 0.1311
 0.1818
 0.0000
 0.0000
 0.0000
 0.0790
 0.1665
 0.1484
 0.9941
 0.0000
 0.1233
 0.0832
 0.9628
 0.3353
 0.2544
 0.9996
 0.9995
 0.9871
 0.1694
 0.9999
 0.2441
 0.0910
 0.9989
 0.0000
[torch.cuda.FloatTensor of size 32 (GPU 0)]

In [154]:
output_gate = g.gate(input_gate)
output_gate=thresh(output_gate,0.25)
output_gate[20]

torch.Size([100, 32])
166
131
145
156
161
145
137
130
153
155
1479


Variable containing:
 0.0000
 0.0000
 0.0000
 0.3505
 0.3077
 0.0000
 0.0000
 0.0000
 0.5120
 0.3664
 0.0000
 0.0000
 0.0000
 0.0000
 0.0000
 0.0000
 0.3919
 0.0000
 0.0000
 0.3965
 0.3540
 0.7096
 0.5414
 0.9204
 0.9587
 0.4407
 0.7560
 0.9893
 0.3430
 0.4772
 0.6195
 0.2822
[torch.cuda.FloatTensor of size 32 (GPU 0)]

In [155]:
noise.data.copy_(fix_noise)
input_main = noise
input_main = input_main.resize(opt.batchSize,opt.nnoise,1,1)                            
        
output = g.main_initial(input_main)    
for i in range(opt.ngres):
    alpha = output_gate[:,i]
    alpha = alpha.resize(opt.batchSize,1,1,1)                                                
    output=g.main[i](output,alpha)
            
output=g.main[opt.ngres](output)                                                          
output=g.main[opt.ngres+1](output)                                                        

save_image(output.data, opt.outf + '/c2.png', nrow=opt.ndiscrete)    